## Notebook to create LDA of 40 topics with speeches

In [3]:
import pandas as pd
import os
from operator import itemgetter
import gensim
import re
from tqdm import tqdm
from os import listdir
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
from collections import Counter

In [4]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

In [68]:
os.chdir('./../../../../../data/02-outcomes/01-policy_positioning/01-text_analysis/wordfish/01_lda/')

## Load Clean Data

In [33]:
def remove_infrequent_words(data):
    remove = list(pd.read_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/wordfish/01_lda/remove_words.csv').remove_words)
    extra_words = ['diputar', 'ley', 
              'articular', 'publicar', 'federal']
    remove  = remove + extra_words
    remove_list = '|'.join(remove)
    regex = re.compile(r'\b('+remove_list+r')\b', flags=re.IGNORECASE)
    
    speech_clean = []
    for x in tqdm(data.clean_speech):
        clean = regex.sub('', x)
        clean = re.sub('[ ]+', ' ', clean)
        speech_clean.append(clean)
    return(speech_clean)

In [34]:
def load_data():
    
    path = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/01-clean_data/disaaggregated_data/'
    data = []
    for i in listdir(path):
        if any(re.findall('json$', i)):
            data.append(pd.read_json(path + i)[['id_speech', 'clean_speech']])
           
    data = pd.concat(data)
    data = data[data['clean_speech'] != '']
    return(data)

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


In [35]:
data = load_data()

In [40]:
data_clean = remove_infrequent_words(data)

100%|██████████| 40414/40414 [4:56:45<00:00,  2.27it/s]   


In [41]:
data['data_clean'] = data_clean

In [43]:
data.to_excel('/Users/cblanesg/Desktop/clean_speaches.xlsx')

In [51]:
import json

In [52]:
with open('/Users/cblanesg/Desktop/clean_speaches.json', 'w') as f:
    json.dump(data.to_dict(), f)

In [58]:
def lda_model_function(num_topics, corpus, id2word):
    print('training model')
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics)
    
    return(lda_model)

In [59]:
def make_bigrams(texts):
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.    
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, nlp):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_])
    return texts_out

In [70]:
def load_corpus(data):
        ## prepare inputs
    print('loding data')
    #data = load_data()
    data = data.data_clean.values.tolist()
    data_words = list(sent_to_words(data))
    
    # Create Dictionary
    print('preparing input')
    id2word = corpora.Dictionary(data_words)

    # Create Corpus
    texts = data_words

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    return(data_words, corpus, id2word)

In [89]:
def obtain_topics_docs(lda_model, corpus):
    
    all_topics = []
    for i in tqdm(corpus):
        main_topic = max(lda_model[i],key=itemgetter(1))[0]  
        all_topics.append(main_topic + 1)
    return(all_topics)
    

In [62]:
def obtain_visualization(num_topics, lda_model, corpus, id2word):
    LDAvis_data_filepath = os.path.join('./lda_visualization/lda_prepared_'+str(num_topics))
    
    if 1 == 1:
        LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
        with open(LDAvis_data_filepath, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)

    # load the pre-prepared pyLDAvis data from disk
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)

    pyLDAvis.save_html(LDAvis_prepared, './lda_visualization/lda_prepared_'+ str(num_topics) +'.html')
    LDAvis_prepared

## LDA

In [71]:
data_words, corpus30, id2word30 = load_corpus(data)

loding data
preparing input


In [72]:
lda_model30k = lda_model_function(30, corpus30, id2word30)

training model


In [113]:
lda_model30k

## Analysis LDA

In [73]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

In [74]:
pyLDAvis.enable_notebook()

In [75]:
obtain_visualization(30, lda_model30k, corpus30, id2word30)

## Check best number of topics

In [138]:
data_text = load_data()

In [90]:
topics30k = obtain_topics_docs(lda_model30k, corpus30)

100%|██████████| 40414/40414 [01:57<00:00, 343.74it/s]


In [91]:
data['topics30k'] = topics30k

In [101]:
data.groupby(['topics30k']).count()

,id_speech,clean_speech,data_clean
topics30k,,,
1,1033,1033,1033
2,808,808,808
3,876,876,876
4,2626,2626,2626
5,1089,1089,1089
6,1151,1151,1151
7,1839,1839,1839
8,544,544,544
9,401,401,401


In [102]:
with open('/Users/cblanesg/Desktop/clean_speaches_topics.json', 'w') as f:
    json.dump(data.to_dict(), f)

In [103]:
data.to_excel('/Users/cblanesg/Desktop/clean_speaches_topics.xlsx')

In [110]:
lda_model30k[corpus30[4]]

[(9, 0.037157632), (16, 0.07184446), (22, 0.88363147)]

In [104]:
data

,id_speech,clean_speech,data_clean,topics30k
0,c722e947-9e5e-3eae-8e47-e696d7d93af3,muchisimas gracia senor presidente companeras ...,muchisimas senor y jefe familia luchar a dific...,1
1,7ef74c2e-c448-3edf-aeae-17c21d0001c9,agradecer presidente compromiso partir politic...,agradecer compromiso politicos agradecerle a j...,5
2,3b25563c-cbf7-39a7-8bde-9aa982717eea,muchisimas gracia venia diputar presidente com...,muchisimas asamblea tribuna posicionamiento re...,7
3,b365b74e-8392-3260-978f-6b1722d4e363,tarde a permiso senora presidente companeras y...,tarde a senora y a nombrar revolucionario inst...,12
4,ed1f1f7e-aeed-3a04-83a2-abe7fc41be06,venia senor presidente companeras diputar comp...,senor alegrar soberania votar unanimidad dict...,23
...,...,...,...,...
8196,b235ddf6-d048-37b8-9bc1-dff953c9ddc9,gracia presidente reservar articular ley siste...,reservar sistema seguridad proponer derogacio...,27
8197,dfdf3a22-e9ff-31c5-940e-9e0dbdacccbd,gracia presidente diputar secretario bienvenid...,secretario bienvenido gustar recibir a mujer ...,27
8198,6a7a24f3-46f3-38a6-b9d0-29848f8fca83,gracia presidente diputar empezar saludar resp...,empezar saludar respetar secretario alfonso d...,27
8199,5f9aced1-def4-3fa7-aac2-6cabace1342b,gracia diputar presidente diputar sumision mie...,sumision miedo silenciar comodidad terminar y...,18
